In [9]:
import h5py
import numpy as np
import scipy as sp

GNewton = 6.67408e-11       #Gravitational constant in SI
GNewton_cgs = 6.67408e-8    #Gravitational constant in cgs
kBoltzmann = 1.3806e-23     #Boltzmann constant in SI
clight = 2.99792458e8       #speed of light in SI
clight_cgs = 2.99792458e10  #speed of light in cgs
Msol = 1.989e30             #Solar mass in SI
permittivity = 8.854187e-12 #permittivity of free space in SI

#internal Cactus conversion factors
cactusM = 5.028916268544129e-34
cactusL = 6.772400341316594e-06
cactusT = 2.0303145448833407e5
cactusV = 1.0/(cactusL*cactusL*cactusL)

Gc2 = GNewton/clight**2*1e-3*Msol
Gc2_cgs = GNewton_cgs/clight_cgs**2
convMsoltokm = Gc2
convMsoltokg = 1.989e30
convkmtos = 1./(clight*1e-3)
convkmtoinvGauss = np.sqrt(permittivity*GNewton/clight**2)*1e3*1e-4
convinvkm2togpercm3=1e-10/Gc2_cgs

convinvm2togpercm3=1e-10/Gc2_cgs*1e-6

convcactustocgs_density=1./(convMsoltokm**2)*convinvkm2togpercm3
convcactustocgs_pressure=1./(convMsoltokm**2)*convinvkm2togpercm3*clight_cgs**2
convcgstocactus_density = cactusM*cactusV
convcgstocactus_pressure = cactusM/(cactusL*cactusT*cactusT)

convGeVtokg = 1.78e-27
convGeVtoinvcm = 5.06e13
convGeVtoinvs = 1.52e24
convGeVtoK =1.16e13
convamutokg =1.6605e-27

convcmtoinvGeV = 5.06e13
convstoinvGeV = 1.52e24

convinvGeVtocm = 1.98e-14
convinvGeVtos = 6.58e-25

convkgtoGeV = 5.62e26
convinvcmtoGeV = 1.98e-14
convinvstoGeV = 6.58e-25
convKtoGeV =8.62e-14
convGeVtoinvFM = 0.0008065*(2*np.pi)*1e-3
convergtoGeV = 624.15

filename = 'eoscompose.h5'

In [35]:
f = h5py.File(filename, 'r')

In [36]:
new_file = 'eoscompose_convertedTEST_B139.h5'

In [37]:
ye = f['Parameters'].get('yq')     # shape == (71) dtype == f8
rho = f['Parameters'].get('nb')    # shape == (114) dtype == f8
temp = f['Parameters'].get('t')    # shape == (34) dtype == f8

EOS = f['Thermo_qty'].get('thermo')                 # shape == (24, 71, 34, 114) dtype == f8
pressure = EOS[0]
entropy_per_baryon = EOS[1]

# only use mu_e, mu_p, set rest to zeros for now
mu_b = EOS[2] # shifted baryon chemical potential mu_b-m_n    [MeV]
mu_e = EOS[3] # charge chemical potential mu_q                [MeV]
mu_p = EOS[3]
mu_l = EOS[4] # lepton chemical potential mu_l                [MeV]
free_E_per_Baryon = EOS[5] # is this the same as internal E/baryon EOS[6]?

EOS_index = f['Thermo_qty'].get('index_thermo')     # shape == 24 dtype == i4

yi = f['Composition_pairs'].get('yi')               # shape == (7, 71, 34, 114) dtype == f8 # relative abundance like electron frac for other isotopes
yi_index = f['Composition_pairs'].get('index_yi')   # shape == (7) dtype == i4

micro = f['Micro_qty'].get('micro')                 # shape == (2, 71, 34, 114) dtype == f8
index_micro = f['Micro_qty'].get('index_micro')     # shape == (1) dtype == i4

aav = f['Composition_quadrupels'].get('aav')            # shape == (1, 71, 34, 114) dtype == f8  'Abar'
index_av = f['Composition_quadrupels'].get('index_av')  # shape == (1) dtype == i4
nav = f['Composition_quadrupels'].get('nav')            # shape == (1, 71, 34, 114) dtype == f8
yav = f['Composition_quadrupels'].get('yav')            # shape == (1, 71, 34, 114) dtype == f8
zav = f['Composition_quadrupels'].get('zav')            # shape == (1, 71, 34, 114) dtype == f8

energy_shift_num = 9.24041641437038*10**18

print(f.keys())

aav_3d = aav[0]            # shape == (71, 34, 114) dtype == f8
nav_3d = nav[0]            # shape == (71, 34, 114) dtype == f8
yav_3d = yav[0]            # shape == (71, 34, 114) dtype == f8
zav_3d = zav[0]            # shape == (71, 34, 114) dtype == f8

Abar = aav_3d
Xa = aav_3d*0
Xh = aav_3d*0
Xn = aav_3d*0
Xp = aav_3d*0
Zbar = aav_3d*0
Cs2 = EOS[11]
dedt = aav_3d*0                 # erg g^-1 K^-1                zeros?
dpderho = EOS[10]               # fm^-3  convert to  g cm^-3        
dpdrhoe = EOS[9]                # MeV convert to cm^2 s^-2
energy_shift = np.array([energy_shift_num])
entropy = EOS[1]
gamma = EOS[14]
logenergy = np.log10(EOS[6] + energy_shift_num)  # change units?  E/mc^2 - 1 
#the log10 of the specific internal energy + the energy shift (to ensure positive energies) in erg/g)

logpress = np.log10(EOS[0]*1.60217733e-6)    # MeV fm^-3 convert to dyn cm^-2 1MeV = 1.60217733E-6 dyn*cm
logrho = np.log10(rho)         # already in MeV   shape should == (114,)
logtemp = np.log10(temp)       # already in MeV   shape should == (34,)
mu_e = EOS[3]                  # already in MeV
mu_n = EOS[3]*0
mu_p = EOS[3]                  # already in MeV
muhat = EOS[2]                 # already in MeV
munu = EOS[3]*0
pointsrho = np.array([len(logrho)])
pointstemp = np.array([len(logtemp)])
pointsye = np.array([len(ye)])
ye = ye        # shape should == (71,)

print('ye shape: ', ye.shape)
print('logrho shape: ', logrho.shape)
print('logtemp shape: ', logtemp.shape)
print('pointstemp shape: ', pointstemp.shape)
print('energy_shift shape: ', energy_shift.shape)
print(Abar.shape)

<KeysViewHDF5 ['Composition_pairs', 'Composition_quadrupels', 'Micro_qty', 'Parameters', 'Thermo_qty', 'metadata']>
ye shape:  (71,)
logrho shape:  (114,)
logtemp shape:  (34,)
pointstemp shape:  (1,)
energy_shift shape:  (1,)
(71, 34, 114)


In [38]:
Parameters = {'Abar': Abar,
              'Xa': Xa,
              'Xh': Xh,
              'Xn': Xn,
              'Xp': Xp,
              'Zbar': Zbar,
              'Cs2': Cs2,
              'dedt': dedt,
              'dpderho': dpderho,
              'dpdrhoe': dpdrhoe,
              'energy_shift': energy_shift,
              'entropy': entropy,
              'gamma': gamma,
              'logenergy': logenergy,
              'logpress': logpress,
              'logrho': logrho,
              'logtemp': logtemp,
              'mu_e': mu_e,
              'mu_n': mu_n,
              'mu_p': mu_p,
              'muhat': muhat,
              'munu': munu, 
              'pointsrho': pointsrho,
              'pointstemp': pointstemp, 
              'pointsye': pointsye, 
              'ye': ye
             }

In [39]:
for i in Parameters:
    try:
        assert Parameters[i].shape == (71, 34, 114)
        print(i)
    except:
        print(i, 'shape:', Parameters[i].shape)
    finally: continue

Abar
Xa
Xh
Xn
Xp
Zbar
Cs2
dedt
dpderho
dpdrhoe
energy_shift shape: (1,)
entropy
gamma
logenergy
logpress
logrho shape: (114,)
logtemp shape: (34,)
mu_e
mu_n
mu_p
muhat
munu
pointsrho shape: (1,)
pointstemp shape: (1,)
pointsye shape: (1,)
ye shape: (71,)


In [41]:

n = h5py.File(new_file, 'w')

for i in Parameters:
    n.create_dataset(i, data=Parameters[i])
    


